###필요한 모듈 import

In [ ]:
!pip install --upgrade openai

In [ ]:
import os
import json
import openai  # GPT API 호출을 위한 라이브러리
import base64

In [ ]:
OPENAI_API_KEY = "YOUR-API-KEY"

###이미지 base64 인코딩 함수

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

###GPT API를 활용하여 이미지 설명 생성하는 함수

In [ ]:
def generate_fashion_description(image_path):

    encoded_image = encode_image(image_path)

    prompt = '''
    You are a creative fashion image captioning assistant. Generate a two-sentence description of a fashion outfit based on the image.

    If shoes or footwear are clearly visible: Describe the footwear factually using this exact format:
      'The outfit is styled with [type of footwear] featuring [notable characteristics].'

    If footwear is not visible or cannot be identified: Skip the first sentence.

    Always include this sentence: Add a stylized, emotional summary using this exact format: 'This [style] look evokes [mood], perfect for [season]’s [setting].'

    To encourage variety and consistency, follow these expression banks:
    Use the [style] to guide the selection of [mood] from its matching group. Do not reuse the same word across multiple generations. Be creative, polished, and fashion-forward.

    [Style → Recommended Mood Group]
    • sexy → Bold & Edgy(bold, fierce, rebellious, confident, daring, sharp)
    • elegant → Chic & Sophisticated(elegant, refined, polished, timeless, understated)
    • casual → Youthful & Playful(playful, spirited, fresh, bubbly, carefree)
    • chic → Chic & Sophisticated / Soft & Serene(dreamy, tranquil, poetic, gentle, graceful, airy)
    • vintage → Romantic & Feminine(romantic, sweet, whimsical, delicate, nostalgic) / Soft & Serene
    • minimalist → Soft & Serene / Chic & Sophisticated
    • street → Bold & Edgy / Youthful & Playful
    • preppy → Youthful & Playful / Romantic & Feminine

    [Setting Bank – time/place/occasion examples]
    daily wear, a casual café outing, a weekend getaway, a beach day, a picnic in the park, a morning run, a mountain hike, a school day, a yoga session, a romantic date, a stroll on campus, a stylish office day, a cozy camping trip, a summer festival, a wedding ceremony, a first date, a casual tennis match, a day at the pool, a confident interview, a formal family gathering

    —
    Example Outputs:
    - The outfit is styled with white two-strap flat sandals featuring a minimal open-toe design. This minimalist look evokes poetic softness, perfect for a summer afternoon gallery stroll.
    - This vintage look evokes whimsical charm, perfect for an early autumn countryside wedding.
    - The outfit is styled with black leather ankle boots featuring a pointed toe and zipper trim. This sexy look evokes fierce confidence, perfect for a rooftop evening soirée.
    - The outfit is styled with off-white ankle boots featuring a matte finish and minimal silhouette. This romantic look evokes vintage whimsy, perfect for a golden-hued autumn bookstore visit.
    '''

    client = openai.OpenAI(api_key=OPENAI_API_KEY)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "This is the fashion image related to the provided metadata."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}}
                ]
            }
        ],
        temperature=1.0,
        max_tokens=100
    )

    return response.choices[0].message.content

###이미지 경로 설정 후 각 폴더 별 n개의 이미지 설명 출력

In [ ]:
base_dir = "./원천데이터/원천데이터_1"
style_folders = ["레트로", "로맨틱", "리조트"]

results = []

for style in style_folders:
    folder_path = os.path.join(base_dir, style)
    all_files = os.listdir(folder_path)

    selected_images = all_files[:10] #각 폴더 별 10개의 이미지 설명 출력

    for image_file in selected_images:
        image_path = os.path.join(folder_path, image_file)
        try:
            description = generate_fashion_description(image_path)
            print(f"\n [{style}] {image_file}")
            print(description)
            results.append((style, image_file, description))
        except Exception as e:
            print(f"❌ Error processing {image_file}: {e}")
